# Algorithme Génétique - BeOrNotToBe
*Matis CAFFIAUX* - Mars 2022

Ce projet a pour but de mettre en application un algorithme génétique (domaine des métaheuristique) sur un cas concret.</br>
Tel qu'Hamlet dans Shakespeare regardant un crâne dans sa paume, disant cette phrase très connue: *"Être où ne pas être, telle est la question"*

## Modèle de donnée
Le modèle utilisé est une chaine de caractère ayant comme gène, chaque lettre de la chaine.
L'ensemble des gènes forme le génotype de l'individu.

## Croisement entre deux individus
Pour le croisement nous effectuons un random sur chaque caractère pour décidé si nous utilisons un gène du Parent 1 (Père) ou du Parent 2 (Mère).

## Remplacement de la Population pour la génération suivante
Nous avons effectué **N** croisement (*Taille de la population x Probabilité de Croissance*), que l'on utilisera pour remplacer les **N** pire individu dans la population.

## Mutation d'un individu
Nous avons décider de effectuer une mutation sur les genes aléatoirement et un nombre aléatoire, avec une borne maximale de la taille du génotype (chaine de caractère).

In [1]:
from operator import ge
from random import randrange , random

# **Individu** (Person) - Classe
Classe permetant d'intancier un individu & effectuer des opérations

## Attribut
1. **Genotype**: chaine de caratères regroupant l'ensemble des gènes
>**Exemple** : "beornottobe", "matis", "algorithm" 

## Methodes
1. **Constructeur** : permet d'intancier un individue aléatoirement avec une **taille définit**, peut prendre une genotype par defaut
>**Exemple** : Person(genotypeLength=13) | individue aléatoire de taille 13
>**Exemple** : Person(genotypeLength=5, genotype="matis ) | individue déterminer ("matis") de taille 5

2. **Operateur d'affichage** : Permet d'afficher le genotype d'un individue
>**Exemple** : print(Person(genotypeLength=5, genotype="matis" )) >>> "matis"

3. **Opérateur d'addition** : Permet de croisser deux instance d'individu et retourne un individu enfant
>**Exemple** : Person(genotypeLength=5, genotype="matis" ) + Person(genotypeLength=5, genotype="sitam) >>> "sitis" type(Person Class) 

4. **Replacement d'un gène** : Permet de changer un gène d'individu aléatoirement à un point précis
>**Exemple** : p = Person(genotypeLength=5, genotype="matis" ).replace(0) >>> print(p) >>> "datis"

5. **Mutation d'un génotype** : Permet de muter le génotype d'individu aléatoirement
>**Exemple** : p = Person(genotypeLength=5, genotype="matis" ).mutate(0) >>> print(p) >>> "gadit"

6. **Evaluation d'une solution** : Permet d'evaluer le individu
>**Exemple** : p = Person(genotypeLength=5, genotype="matis" ).evaluate("matis") >>> 1.0


In [2]:
class Person() :

    genotype : str = ""

    def __init__(self, genotypeLength : int, genotype="") -> None:
        if genotype == "":
            for _ in range(genotypeLength):
                self.genotype +=  (lambda : chr(ord("a") +randrange(25)))()
        else:
            self.genotype = genotype

    def __str__(self) -> str:
        return self.genotype

    def __add__(self, other): # Multi-cross random points
        childGenotype = ""
        
        for index in range(len(self.genotype)):
            
            if randrange(2) == 0:   # Left Parent
                childGenotype += self.genotype[index]
            else:                   # Right Parent
                childGenotype += other.genotype[index]
    
        return Person(len(self.genotype),childGenotype)

    def replace(self , toRemove:int) -> None:
        temp = ""
        for index, elem in enumerate(self.genotype):
            if index != toRemove:
                temp += elem
            else:
                temp += (lambda : chr(ord("a") +randrange(25)))()
        self.genotype = temp

    def mutate(self):
        mutation = randrange(len(self.genotype))
        for mutation in range(mutation):
            self.replace(randrange(len(self.genotype)))

    
    def evaluate(self, genotypeGoal : str) -> float:
        temp : float = 0.0

        for index , elem in enumerate(self.genotype):
            if elem == genotypeGoal[index] : 
                temp += 1

        return temp/(index+1)

# **Population** (Population) - Classe
Classe rassemblant un ensemble d'individu 

## Attributs
1. **Individue de la population**: Ensemble d'individu dans une liste
>**Exemple** : ["boornottobe", "beornottobe", ... ]

2. **Probabilité de Croissance**: Nombre d'individu créé pour la génération suivante
>**Exemple** : défaut : 0.5

3. **Probabilité de Mutation**: Probalilité qu'un individu effectue une mutation 
>**Exemple** : defaut: 0.3

4. **Objectif**: Génotype *"d'objectif"* permetant d'evaluer les individus  

## Methodes
1. **Constructeur** : permet d'intancier un ensemble individue aléatoirement avec une **taille définit**

2. **Operateur d'affichage** : Permet d'afficher les génotypes des individus et leurs evaluations

3. **Tri des individus** : Permet de trier la population d'individus 

4. **Nouvelle Génération** : Permet de passer la population sur une nouvelle génération 

5. **Iterer des Nouvelles Générations** : Permet d'iterer n-nouvelle générations

6. **Jusqu'a la solution** : Permet d'iterer jusuqu'a la solution

In [3]:
class Population():
    pop: list = []

    probGrowth : float
    probMutation : float

    genotypeGoal :str 

    def __init__ (self , sizeOfPop : int ,genotypeLength : int, genotypeGoal: str , probGrowth : float = 0.50, probMutation : float = 0.30) -> None :
        self.probGrowth = probGrowth
        self.probMutation = probMutation
        self.genotypeGoal = genotypeGoal
        
        for _ in range(sizeOfPop):
            self.pop.append(Person(genotypeLength))
        
    def __str__(self) -> str:
        temp = ""
        for index, elem in enumerate(self.pop):
            temp+= f"Person {index}: "+elem.__str__() +" "+str(elem.evaluate(self.genotypeGoal))+"\n"
        return temp

    def sort(self) -> None:
        self.pop = sorted(self.pop, key=lambda x: x.evaluate(self.genotypeGoal), reverse=True)
    

    def newGen(self) -> None:
        self.sort()

        reproducers = self.pop[0:len(self.pop)//6]
        childs = []

        i,j = 0,1
        while(len(childs)< int(len(self.pop)*self.probGrowth)):
            if(i == len(reproducers)):
                i, j = 0 , j+1

            child = reproducers[i]+reproducers[j]

            if(random()<=self.probMutation): # Mutation
                child.mutate()

            childs.append(child)

        self.pop[len(self.pop)-1 - int(len(self.pop)*self.probGrowth):-1] = childs # Replacement of worst parents

    def iterate(self, nIter : int) -> None:
        for _ in range(nIter):
            if(_%((nIter//10)+1) == 0):

                print(f"Iteration n°:{_} \r")
            self.newGen()

            if(self.pop[0].evaluate(self.genotypeGoal) ==1.0):
                print(f"Goal has been found in {_} Iterations \n")
                return None
    
    def until(self):
        nInter = 1
        while True:
            if(nInter%100 == 0):
                print(f"Iteration n°:{nInter} \r", )

            self.newGen()
            nInter+=1

            if(self.pop[0].evaluate(self.genotypeGoal) == 1.0):
                print(f"Goal has been found in {nInter} Iterations \n")
                return None

In [4]:
if __name__ == '__main__':
    pop = Population(2000, 11, "beornottobe")
    pop.until()
    print(pop.pop[0], pop.pop[0].evaluate("beornottobe"))
    


Goal has been found in 8 Iterations 

beornottobe 1.0
